In [1]:
import pandas as pd
import numpy as np
from script import *
import itertools
import lightgbm as lgbm

In [2]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','size']

In [15]:
start = 20130101
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
print(start_date)
# df_r = pd.read_parquet(r"database\return_8%_20%_60_20")
# df_rs = pd.read_parquet(r"database\return_5%_10%_20_8_inf%").loc[idx[:,start_date:],:].sort_index()
# df_rs_sp = pd.read_parquet(r"database\return_5%_10%_20_8_15% v0").loc[idx[:,start_date:],:].sort_index()
df_rs_sp2 = pd.read_parquet(r"database\return_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()

2013-01-01


In [18]:
# inter_index = df_rs_sp.index.intersection(df_rs.index) # 避免范围不同导致的差别
if len(inter_index)==len(df_rs_sp)==len(df_rs):
    res = pd.DataFrame([df_rs_sp2.loc[:,'r'].agg(agg_operations[:4]),
                    df_rs_sp.loc[:,'r'].agg(agg_operations[:4]),
                    df_rs.loc[:,'r'].agg(agg_operations[:4])],
                   index=['stop_profit2','stop_profit',"not_stop_profit"]).T
else:
    res = pd.DataFrame([df_rs_sp.loc[inter_index,'r'].agg(agg_operations[:4]),
                    df_rs.loc[inter_index,'r'].agg(agg_operations[:4])],
                   index=['stop_profit',"not_stop_profit"]).T
res['sp/|nsp|-1'] = (res['stop_profit']-res['not_stop_profit'])/res['not_stop_profit'].abs()*100
res['sp2/|nsp|-1'] = (res['stop_profit2']-res['not_stop_profit'])/res['not_stop_profit'].abs()*100

# print("不止盈的收益统计",df_rs.r.agg(agg_operations[:4]))
# print("止盈的收益统计",df_rs_sp.loc[inter_index,'r'].agg(agg_operations[:4]))
print(res)

# 结论：
# 偏短模式下15%收盘次日开盘止盈，mean提升了10.68%，提升明显，
#   表明止盈减少的利润回撤远大于放弃的进一步涨幅，大部分个股短期涨幅在15%附近，继续大涨的个股比例较小。
# median提升了0.39%，推测是中位数以上的赚钱样本提升明显，但中位数以下基本达不到止盈条件，相当于不变，所以中位数最终提升轻微。

        stop_profit2  stop_profit  not_stop_profit  sp/|nsp|-1  sp2/|nsp|-1
mean        0.002697     0.002797         0.002754    1.543012    -2.083758
median     -0.008584    -0.008565        -0.008569    0.046551    -0.167942
max        10.329513    10.329513        10.329513    0.000000     0.000000
min        -0.872727    -0.872727        -0.872727    0.000000     0.000000


In [14]:
print(df_rs_sp.loc[idx[:,'2016-01-01':'2020-03-19'],'r'].agg(agg_operations))
print(df_rs.loc[idx[:,'2016-01-01':'2020-03-19'],'r'].agg(agg_operations))

mean     -2.585487e-04
median   -1.063830e-02
max       5.111111e+00
min      -8.727273e-01
size      1.359186e+06
Name: r, dtype: float64
mean     -2.572295e-04
median   -1.063830e-02
max       5.111111e+00
min      -8.727273e-01
size      1.359186e+06
Name: r, dtype: float64
